In [ ]:
import sys
sys.path.append('../utils')
sys.path.append('../core')
sys.path.append('../')
import numpy as np
import scipy
import tqdm
import utils
import geometry
from core import *

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import glob


In [ ]:

test_phantom = phantom.Phantom(source_path = None,
            voxel_dims = (0.1e-3,0.1e-3,0.1e-3), # size of each voxel
            matrix_dims = (400,400,400), # number of voxels along each axis
            baseline = (1500, 1000), # background medium speed of sound and density
            seed = 5678,)

kidney_sos = 1560
kidney_rho = 1066
kidney_sigma = 27
kidney_noise_scale = 0.0003
fat = tissue.Tissue(name='fat', c=1450, rho=920, sigma=10, scale=0.0001, label=1)
kidney = tissue.Tissue(name='kidney', c=kidney_sos, rho=kidney_rho, sigma=kidney_sigma, 
                scale=kidney_noise_scale, label=2) 

pos = 0.4

test_phantom.add_tissue_sphere((pos,-pos,0), 50, fat) # sphere center, radius, tissue makeup
test_phantom.add_tissue_sphere((-pos,pos,0), 50, kidney)

plt.imshow(test_phantom.get_complete()[0,:,:,test_phantom.matrix_dims[2]//2], 
           vmin=np.amin(test_phantom.get_complete()[0]), vmax=np.amax(test_phantom.get_complete()[0]))
plt.colorbar()

n_transducers = 1 # number of transducers in trajectory

transducers = [transducer.Planewave(max_frequency = 2000000, # [Hz],
                         elements = 32,
                         element_width = 1e-4, # [m]
                         elevation = 1e-4 * 32, 
                         sweep = np.pi/3, # angles from -60 to 60 
                         ray_num = 64, # number of steering angles in the range prescribed by sweep 
                         focus_elevation = 0.02,# wherever you'd like to put the elevation focus
                         sensor_sampling_scheme = 'not_centroid',
                         )
                for i in range(n_transducers)]

for t in transducers:
    t.make_sensor_coords(test_phantom.baseline[0])

test_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)

# global coordinate system origin (0,0,0) at centroid of phantom as defined above
for i in range(n_transducers):
    transducer_transform = geometry.Transform(translation=(0,0,0), rotation=(0,0,0)) # global coords of transducer centroid, zyx intrinsic tait-bryan
    test_transducer_set.assign_pose(i, transducer_transform)

test_transducer_set.plot_transducer_fovs(scale=0.01)
test_transducer_set.plot_transducer_coords(scale=0.01)

test_sensor = sensor.Sensor(transducer_set=test_transducer_set, aperture_type='single_transducer')

simprops = simulation.SimProperties(grid_size   = (1024,128,128), # you may need to tweak the grid size for maximum efficency
                 PML_size    = (32,8,8),
                 PML_alpha   = 2,
                 t_end       = 12e-5,           # [s]
                 bona        = 6,               # parameter b/a determining degree of nonlinear acoustic effects
                 alpha_coeff = 0.5, 	        # [dB/(MHz^y cm)]
                 alpha_power = 1.5,
                 )

In [ ]:
test_experiment = experiment.Experiment(
                 simulation_path = '../experiment_files/april_test', # your desired path
                 sim_properties  = simprops,
                 phantom         = test_phantom,
                 transducer_set  = test_transducer_set,
                 sensor          = test_sensor,
                 nodes           = 2, # how many compute nodes you plan to use
                 results         = None,
                 indices         = None,
                 workers         = 3, # how many workers/node
                 )

test_experiment.save()

test_experiment.run(dry=True)

#test_experiment.add_results()

#reconstruction = Compounding(test_experiment) # beta, may not currently work as intended




In [ ]:
test_experiment = experiment.Experiment.load('../experiment_files/april_test')

In [ ]:
test_experiment.run()